<img src="assets/logo.png" alt="TinkerTales Storymaker Logo" width="300"/>

# 📖✨ TinkerTales Storymaker ✨📖  
*“Where imagination meets AI and comes to life.”*


# TinkerTales Storymaker

This notebook generates an original children's story using OpenAI, narrates it with a theme-appropriate voice using ElevenLabs,  
and saves the story as an audio file (.mp3), plain text (.txt), and structured metadata (.json).

---

### 🔹 Features
- Story tone and voice are matched to theme and age group
- Narration is saved using ElevenLabs voice synthesis
- Output files are saved in a `narrated_stories/` directory

### 🔧 Technologies Used
- Python
- OpenAI GPT-4o for story generation
- ElevenLabs API for voice synthesis
- dotenv for secure API key handling
- JSON and text file output

### 🌱 Planned Features
- AI-generated illustrations
- Sound effect integration
- Interactive web interface (e.g., Streamlit or Gradio)

*Author: Talia Chakraborty*

In [ ]:
print("Welcome to TinkerTales Storymaker!")

In [ ]:
import os
import json
from dotenv import load_dotenv
from elevenlabs import ElevenLabs, VoiceSettings
from openai import OpenAI

# 🔐 Load environment variables
load_dotenv()

# OpenAI client (for story generation)
openai_client = OpenAI()  # uses your OPENAI_API_KEY from .env

# ElevenLabs client (for narration)
elevenlabs_client = ElevenLabs(api_key=os.getenv("ELEVEN_API_KEY"))

In [ ]:
def narrate_story(story_text, filename="story.mp3", voice="Amelia"):
    audio = elevenlabs_client.text_to_speech.convert(
        voice=voice,
        model="eleven_monolingual_v1",
        text=story_text,
        voice_settings=VoiceSettings(stability=0.7, similarity_boost=0.8)
    )
    with open(filename, "wb") as f:
        f.write(audio)
    print(f"🎧 Narration saved as {filename}")

In [ ]:
def select_voice(theme, age_range):
    """
    Returns the best ElevenLabs voice based on story theme and age range.
    Legacy voices are only used as optional extras.
    """
    if theme == "Bedtime":
        return "Charlotte"
    
    elif theme == "Fairy Tale":
        return "Amelia"  # or "Lily"

    elif theme == "Spooky":
        return "Daniel"  # Adam Stone as backup

    elif theme == "Comedy":
        if age_range == "6-8":
            return "Jessica"  # energetic, playful
        elif age_range == "9-11":
            return "Frederick Surrey"  # clever, expressive
        else:
            return "Amelia"  # safe default

    elif theme == "Fantasy":
        return "Frederick Surrey"  # rich and imaginative

    elif theme == "Adventure":
        return "Alice"  # curious and upbeat

    elif theme == "Mystery":
        return "Adam Stone"

    elif theme == "Outer Space":
        return "Adam Stone"  # or "Alice" for more whimsical

    elif theme == "Science Fiction":
        return "Jessica"

    else:
        return "Amelia"  # default fallback

In [ ]:
# Style mapping for themes (except Comedy and Adventure)
style_by_theme = {
    "Fairy Tale": "in a whimsical, magical style like J.K. Rowling",
    "Bedtime": "in a gentle, soothing voice like Margaret Wise Brown",
    "Outer Space": "in a blend of cosmic wonder and humor, like Carl Sagan meets Roald Dahl",
    "Science Fiction": "in a curious, thoughtful voice like Madeleine L'Engle",
    "Fantasy": "in a mythic, magical tone like C.S. Lewis",
    "Mystery": "in a clever, dramatic style like Lemony Snicket",
    "Spooky": "in a slightly eerie but fun tone like R.L. Stine"
}

# Styles for Comedy and Adventure based on age range
comedy_style_by_age = {
    "3-5": "in a silly, rhythmic style like Sandra Boynton",
    "6-8": "in a wacky, outrageous style like Dav Pilkey (Dog Man)",
    "9-11": "in a clever, humorous tone like Roald Dahl"
}

adventure_style_by_age = {
    "3-5": "in a rhythmic, playful style like Julia Donaldson or Mo Willems",
    "6-8": "in a rhyming, action-packed style like Julia Donaldson",
    "9-11": "in a fast-paced, witty tone like Rick Riordan"
}

In [ ]:
def generate_story(character_name, age_range, theme, custom_detail=None):
    age_sensitive_themes = ["Comedy", "Adventure"]

    if theme in age_sensitive_themes:
        if theme == "Comedy":
            style = comedy_style_by_age.get(age_range, "")
        elif theme == "Adventure":
            style = adventure_style_by_age.get(age_range, "")
    else:
        style = style_by_theme.get(theme, "")

    prompt = f"""
    Write a short story for a child aged {age_range}. The story should include a character named {character_name} and follow the theme "{theme}".
    {f"Include this detail: {custom_detail}." if custom_detail else ""}
    Write the story {style}.
    Include a creative, fun story title at the top.
    The story should have a clear beginning (introducing the character and setting), middle (a challenge or adventure), and end (a satisfying resolution).
    Make the story around 500-650 words long. Keep it age-appropriate and imaginative.
    """

    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8,
        max_tokens=1000
    )

    return response.choices[0].message.content

In [ ]:
# Set story parameters
character_name = "Ani"
age_range = "9-11"
theme = "Fantasy"
custom_detail = "Ani wears a shirt with stars on it and loves the dark night sky outside her house. She and her little sister Uma are going on a wild adventure."

# Generate the story
story = generate_story(
    character_name=character_name,
    age_range=age_range,
    theme=theme,
    custom_detail=custom_detail
)

# Print the story in the notebook
print(story)

# Select voice based on theme and age
voice = select_voice(theme, age_range)

# 💾 Prepare filenames and folder
output_dir = "narrated_stories"
os.makedirs(output_dir, exist_ok=True)
base_filename = f"{character_name.lower()}_{theme.lower().replace(' ', '_')}"

# 🎧 Save narration
mp3_filename = os.path.join(output_dir, f"{base_filename}.mp3")
narrate_story(story, filename=mp3_filename, voice=voice)

# 📄 Save story as plain text
txt_filename = os.path.join(output_dir, f"{base_filename}.txt")
with open(txt_filename, "w") as f:
    f.write(story)
print(f"📝 Story text saved as {txt_filename}")

# 📦 Save story metadata as JSON
json_data = {
    "character_name": character_name,
    "age_range": age_range,
    "theme": theme,
    "custom_detail": custom_detail,
    "voice": voice,
    "story_text": story
}
json_filename = os.path.join(output_dir, f"{base_filename}.json")
with open(json_filename, "w") as f:
    json.dump(json_data, f, indent=2)
print(f"📦 Story metadata saved as {json_filename}")


**The Star-Touched Adventure**

In the quaint village of Luminara, where the sky twinkled like a sea of diamonds every night, there lived two spirited sisters, Ani and Uma. Ani, the elder, always wore a shirt adorned with shimmering stars, a tribute to her love for the night sky. Uma, her little sister, followed her everywhere, her heart full of wonder and curiosity.

One starlit evening, as the sisters sat beneath the ancient oak tree outside their house, Ani spotted something unusual. "Look, Uma," she exclaimed, pointing upwards. There, threading through the constellations, was a shooting star, blazing a path across the heavens.

But this was no ordinary shooting star. It suddenly paused mid-sky, shimmering brightly above them. "It’s calling to us!" Ani whispered, her eyes wide with excitement.

The star lowered itself gently, transforming into a glowing bridge leading up into the sky. Uma, clutching Ani’s hand, felt her heart leap with a mix of fear and thrill. "Shall we?" she asked